In [9]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from geopedestrian.categories import CategoryRegistry
from geopedestrian.services import drive_from, solve_walking, walk_from, walk_to
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
import pandas as pd
import requests

In [2]:
host = 'geopedestrian.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Query a defined set of Berlin stations as GeoJSON

In [5]:
def get_berlin_stations():
    raw_url = 'https://gist.githubusercontent.com/gisfromscratch/3535a73c806d42a28a0eab6962f552d8/raw/eaaad5f7598688c2ec2c753e6cffe54d8d12bf16/berlin_starting_locations.geojson'
    geojson_response = requests.get(raw_url)
    geojson_response.raise_for_status()
    return FeatureSet.from_geojson(geojson_response.json())

In [11]:
berlin_stations = get_berlin_stations()
locations_sdf = berlin_stations.sdf
locations_sdf

/mnt/data/miniconda3/envs/geourban-int/lib/python3.9/site-packages/arcgis/features/feature.py:1014: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  warn(


,name,OBJECTID,SHAPE
0,U-Bahn Ruhleben,1,"{""x"": 13.241183855834903, ""y"": 52.525492469777..."
1,U-Bahn Olympiastadion,2,"{""x"": 13.249940139300833, ""y"": 52.516993066967..."
2,S-Bahn Olympiastadion,3,"{""x"": 13.241096425470483, ""y"": 52.511158549082..."
3,S-Bahn Pichelsberg,4,"{""x"": 13.22863185816456, ""y"": 52.5098115984010..."
4,S-Bahn Stresow,5,"{""x"": 13.20952081689347, ""y"": 52.5320197242490..."


## Visualize the walking areas from every defined station

In [10]:
def solve_walking_areas(client: GeoRapidClient, locations_sdf):
    """
    Solve walking areas which are accessible by pedestrians using the specified locations.
    """
    walking_results = locations_sdf.SHAPE.apply(lambda shape: solve_walking(client, shape.y, shape.x))
    walking_areas_sdf = None
    for walking_result in walking_results:
        if None is walking_areas_sdf:
            walking_areas_sdf = FeatureSet.from_geojson(walking_result).sdf
        else:
            walking_areas_sdf = pd.concat([walking_areas_sdf, FeatureSet.from_geojson(walking_result).sdf], ignore_index=True)
    return walking_areas_sdf

In [7]:
gis = GIS()

In [69]:
def plot_map(walking_areas_sdf):
    berlin_map = gis.map('Berlin, Germany')
    berlin_map.basemap = 'osm'
    walking_areas_sdf.spatial.plot(berlin_map, renderer_type='u', col='ToBreak', cmap='RdYlGn', alpha=0.35)
    berlin_map.zoom_to_layer(walking_areas_sdf)
    return berlin_map

In [12]:
walking_areas_sdf = solve_walking_areas(client, locations_sdf)
walking_areas_sdf

,ObjectID,FacilityID,Name,FromBreak,ToBreak,Shape_Length,Shape_Area,SHAPE
0,1,1,Location 1 : 10 - 15,10,15,0.15740288767591595,0.00016789873778614772,"{""rings"": [[[13.230686410000033, 52.5317930550..."
1,2,1,Location 1 : 5 - 10,5,10,0.08936868406718201,9.211986361839968e-05,"{""rings"": [[[13.232707610000034, 52.5297718540..."
2,3,1,Location 1 : 0 - 5,0,5,0.029241417552648863,4.402991422438283e-05,"{""rings"": [[[13.238546633000055, 52.5281998100..."
3,1,1,Location 1 : 10 - 15,10,15,0.16790349871280333,0.00022801747248135827,"{""rings"": [[[13.245733122000047, 52.5268523430..."
4,2,1,Location 1 : 5 - 10,5,10,0.10435066579794955,0.00010962087535717955,"{""rings"": [[[13.246406855000032, 52.5237082540..."
5,3,1,Location 1 : 0 - 5,0,5,0.028226565077630418,3.749855907483814e-05,"{""rings"": [[[13.247529744000076, 52.5201150090..."
6,1,1,Location 1 : 10 - 15,10,15,0.18724654855945144,0.00022743746386415066,"{""rings"": [[[13.24101698800007, 52.52056416500..."
7,2,1,Location 1 : 5 - 10,5,10,0.10848226272188806,0.00012989582042806838,"{""rings"": [[[13.243038188000071, 52.5174200760..."
8,3,1,Location 1 : 0 - 5,0,5,0.03363331393429591,5.338564528342173e-05,"{""rings"": [[[13.242813611000031, 52.5142759870..."
9,1,1,Location 1 : 10 - 15,10,15,0.17631942308922788,0.0002092051545174546,"{""rings"": [[[13.234279654000034, 52.5192166980..."


In [70]:
berlin_walking_maps = [plot_map(walking_areas_sdf.iloc[i:i+3]) for i in range(0, len(walking_areas_sdf), 3)]

In [71]:
berlin_walking_maps[0]

MapView(layout=Layout(height='400px', width='100%'))

In [72]:
berlin_walking_maps[1]

MapView(layout=Layout(height='400px', width='100%'))

In [73]:
berlin_walking_maps[2]

MapView(layout=Layout(height='400px', width='100%'))

In [74]:
berlin_walking_maps[3]

MapView(layout=Layout(height='400px', width='100%'))

In [75]:
berlin_walking_maps[4]

MapView(layout=Layout(height='400px', width='100%'))